<br>

# Introdução

- https://datageo.ambiente.sp.gov.br/navegar

In [1]:
#!pip3 install traquitanas --upgrade

In [2]:
import os
import sys
import time
import math
import pandas as pd
from itertools import islice

In [3]:
from traquitanas import net

In [4]:
from selenium.webdriver.common.by import By

In [5]:
sys.path.append(os.path.join(os.getcwd(), '..', 'src'))
from paths import *

<br>

## Driver

In [6]:
driver = net.scraping.create_driver(
    data_path,
    headless=True,
    adds_path=os.path.join('..', 'scrapy', 'adds'),
    log_path=os.path.join('..', 'scrapy', 'logs')    
)

<br>

## URL

In [7]:
url = 'https://datageo.ambiente.sp.gov.br/geoportal/catalog/search/browse/browse.page?'
driver.get(url)

<br>

## Assuntos

In [8]:
def get_subjects():
    # Abre Catálogo
    time.sleep(2)
    try: driver.find_element(by=By.XPATH, value='//*[contains(@class,"dijitTreeExpando dijitTreeExpandoClosed")]').click()
    except: pass

    # Abre Classificação de Metadados
    time.sleep(2)
    try: driver.find_element(by=By.XPATH, value='//*[contains(@class,"dijitTreeExpando dijitTreeExpandoClosed")]').click()
    except: pass

    dict_subjects = {}
    list_tree = driver.find_elements(by=By.XPATH, value='//*[@id="browse-toc"]//*[@class="dijitTreeLabel"]')
    for i in list_tree:
        subject = i.text
        if subject != '' and subject != 'Catálogo' and subject != 'Classificação do Metadado':
            dict_subjects[subject] = i
    return dict_subjects

<br>

## Pages

In [9]:
def get_pages():
    time.sleep(2)
    start_index = int(driver.find_element(by=By.XPATH, value='//*[@id="browse-results-body"]//*[@id="startIndex"]').get_attribute('value'))
    items_per_page = int(driver.find_element(by=By.XPATH, value='//*[@id="browse-results-body"]//*[@id="itemsPerPage"]').get_attribute('value'))
    total_results = int(driver.find_element(by=By.XPATH, value='//*[@id="browse-results-body"]//*[@id="totalResults"]').get_attribute('value'))
    n_pages = math.ceil(total_results/items_per_page)
    dict_results = {
        'start_index': start_index,
        'items_per_page': items_per_page,
        'total_results': total_results,
        'n_pages': n_pages,
    }
    return dict_results

<br>

## Get Infos from Itens

In [10]:
def get_infos_item(item, subject):
    # Título
    titulo = item.find_element(by=By.XPATH, value='.//*[@class="title"]').text

    # Código
    id_lyr = item.find_element(by=By.XPATH, value='.//*[@class="title"]').get_attribute('id')

    # Resumo
    resumo = item.find_element(by=By.XPATH, value='.//*[@class="abstract"]').text

    dict_links = {
            'Assunto': subject,
            'Titulo': titulo,
            'id_lyr': id_lyr,
            'Resumo': resumo,
            'Carregar no Visualizador': '',
            'Metadados': '',
            'XML': '',
    }

    # Links
    list_links = item.find_elements(by=By.XPATH, value='.//*[@class="links"]//*')
    for link in list_links:
        #
        link_text = link.text
        link_url = link.get_attribute('href')

        # Loop    
        if link_text == 'Metadados':        
            # Input dos caminhos para os metadados
            url_api = 'http://datageo.ambiente.sp.gov.br/geoportal/catalog/search/resource/details.page?uuid='
            link_url = '{}{}'.format(url_api, '%7B{}%7D'.format(id_lyr.replace('{', '').replace('}', '')))
        dict_links.update({link_text: link_url})
    return dict_links

In [11]:
#subject = 'Clima'
#get_infos_item(dict_subjects[subject], subject)

<br>

## Get All Itens

In [12]:
def get_all_itens(n_pages, subject):
    # Vai para a página 1
    try: driver.find_element(by=By.XPATH, value='//*[@id="browse-results-header-pageControl"]//*[contains(text(),"Primeiro")]').click()
    except: pass
    try: driver.find_element(by=By.XPATH, value='//*[@id="browse-results-header-pageControl"]//*[contains(text(),"1")]').click()
    except: pass
    time.sleep(3)

    # Para cada página
    list_itens_infos = []
    for i in range(0, n_pages):
        # Clicar no número da página
        item_n = i + 1
        driver.find_element(by=By.XPATH, value='//*[@id="browse-results-header-pageControl"]//*[contains(text(),"{}")]'.format(item_n)).click()
        time.sleep(1)

        # Pega a lista de itens
        list_itens = driver.find_elements(by=By.XPATH, value='//*[@id="browse-results-body"]//*[@class="snippet"]')

        # Pega infos dos itens
        for item in list_itens:
            list_itens_infos.append(get_infos_item(item, subject))
    
    return list_itens_infos

<br>

# Run

In [13]:
dict_subjects = get_subjects()
#dict_subjects = dict(islice(dict_subjects.items(), 2, 4))
dict_subjects

{'Abastecimento de Água': <selenium.webdriver.remote.webelement.WebElement (session="85096321-d037-43f6-b56f-f73f54fff759", element="1f5f7bd3-54e2-4b99-b073-ff391ba3f4d7")>,
 'Áreas com Diretrizes Especiais de Uso': <selenium.webdriver.remote.webelement.WebElement (session="85096321-d037-43f6-b56f-f73f54fff759", element="a2a921f6-bdce-4f5f-bc3e-bbd6b01f6e9d")>,
 'Áreas Protegidas': <selenium.webdriver.remote.webelement.WebElement (session="85096321-d037-43f6-b56f-f73f54fff759", element="5b284c9f-c0ac-4bf8-97c1-127af0ff03bb")>,
 'Biodiversidade': <selenium.webdriver.remote.webelement.WebElement (session="85096321-d037-43f6-b56f-f73f54fff759", element="4a8afb0a-cafd-4db0-ac91-518455791d3b")>,
 'Clima': <selenium.webdriver.remote.webelement.WebElement (session="85096321-d037-43f6-b56f-f73f54fff759", element="36a64aed-d128-49e4-9489-dc91ac1395e4")>,
 'Controle Ambiental': <selenium.webdriver.remote.webelement.WebElement (session="85096321-d037-43f6-b56f-f73f54fff759", element="506063b2-1d5

In [14]:
dfs = []
# Para cada Assunto
for subject in dict_subjects:
    # Vai para o Assunto
    dict_subjects[subject].click()

    # Pega informações da Página
    info_page = get_pages()
    n_pages = info_page['n_pages']
    print('Assunto: "{}" tem {} páginas de itens, totalizando {} itens'.format(subject, n_pages, info_page['total_results']))

    # Adiciona Dados
    temp = get_all_itens(n_pages, subject)
    dfs.append(pd.DataFrame(temp))

print('Fim')

Assunto: "Abastecimento de Água" tem 13 páginas de itens, totalizando 130 itens
Assunto: "Áreas com Diretrizes Especiais de Uso" tem 50 páginas de itens, totalizando 493 itens
Assunto: "Áreas Protegidas" tem 21 páginas de itens, totalizando 206 itens
Assunto: "Biodiversidade" tem 13 páginas de itens, totalizando 126 itens
Assunto: "Clima" tem 1 páginas de itens, totalizando 7 itens
Assunto: "Controle Ambiental" tem 26 páginas de itens, totalizando 253 itens
Assunto: "Energia" tem 2 páginas de itens, totalizando 12 itens
Assunto: "Fauna" tem 6 páginas de itens, totalizando 56 itens
Assunto: "Fiscalização Ambiental" tem 34 páginas de itens, totalizando 336 itens
Assunto: "Geologia e Recursos Minerais" tem 2 páginas de itens, totalizando 18 itens
Assunto: "Geomorfologia" tem 45 páginas de itens, totalizando 445 itens


StaleElementReferenceException: Message: The element reference of <a href="/app?title=IPT+-+Suscetibilidade+a+inunda%C3%A7%C3%B5es+do+munic%C3%ADpio+de+Pedreira&uuid=%7B6E772F98-A984-4ED3-A32F-D2ED5635B3C3%7D&layer=Susc_Inund_Pedreira&resource=wms%3Ahttp%3A%2F%2Fgeo.ipt.br%3A8080%2Fgeoserver%2Fiptwms%2Fows%3FSERVICE%3DWMS%26"> is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been refreshed
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:183:5
StaleElementReferenceError@chrome://remote/content/shared/webdriver/Errors.jsm:464:5
element.resolveElement@chrome://remote/content/marionette/element.js:681:11
evaluate.fromJSON@chrome://remote/content/marionette/evaluate.js:254:26
evaluate.fromJSON/<@chrome://remote/content/marionette/evaluate.js:245:38
evaluate.fromJSON@chrome://remote/content/marionette/evaluate.js:245:20
evaluate.fromJSON@chrome://remote/content/marionette/evaluate.js:262:

In [ ]:
# Results
df = pd.concat(dfs)
df.to_csv(os.path.join(output_path_tab, 'tab_itens.csv'), index=False)
df.head()

In [ ]:
driver.quit()